In [13]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [5]:
pred = np.load('TCNpred.npy')
real = np.load('TCNtest.npy')

In [4]:
pred.shape

(3422, 6, 12)

In [7]:
def get_dataset_multi_step(Source, scope, train_test_split, window_size, n_out):
    split = int(train_test_split * scope)
    df = Source
    scaler = MinMaxScaler()
    data = scaler.fit_transform(df.values)
    label_list = np.zeros(shape=(scope - window_size - n_out, n_out, df.shape[-1]))

    for i in range(scope - window_size - n_out):
        label_list[i, :] = data[i + window_size:i + window_size + n_out, :]

    X = np.zeros(shape=(scope - window_size - n_out, window_size, data.shape[-1]))

    for i in range(scope - window_size - n_out):
        X[i, :] = data[i:i + window_size, :]

    train_X = X[:split, :]
    train_label = label_list[:split, :]
    test_X = X[split:scope, :]
    test_label = label_list[split:scope, :]
    return train_X, train_label, test_X, test_label, scaler

In [9]:
window_size = 72
pred_len = 6
df = pd.read_csv('./new.csv')
df = df.drop(['Unnamed: 0', 'Time'], axis=1)
feature_size = df.shape[1]
Savepath = './new'
train_X, train_label, test_X, test_label, scaler = get_dataset_multi_step(df, 35000, 0.9,
                                                                          window_size=window_size,
                                                                          n_out=pred_len)

In [10]:
from utils.metrics import metric

In [15]:
mae_list = []
rmse_list = []
for i in tqdm(range(pred.shape[0])):
    pred_temp = pred[i, :, :]
    real_temp = real[i, :, :]
    inverse_pred = scaler.inverse_transform(pred_temp)
    inverse_real = scaler.inverse_transform(real_temp)
    mae, mse, rmse, mape, mspe = metric(inverse_pred, inverse_real)
    mae_list.append(mae)
    rmse_list.append(rmse)






 15%|█▌        | 515/3422 [00:00<00:01, 1698.92it/s]/home/Tabe/Documents/Pro/Mpnn_trans_demo/utils/metrics.py:41: RuntimeWarning: divide by zero encountered in true_divide
  return SSReg / SST
/home/Tabe/Documents/Pro/Mpnn_trans_demo/utils/metrics.py:41: RuntimeWarning: divide by zero encountered in true_divide
  return SSReg / SST
/home/Tabe/Documents/Pro/Mpnn_trans_demo/utils/metrics.py:41: RuntimeWarning: divide by zero encountered in true_divide
  return SSReg / SST
/home/Tabe/Documents/Pro/Mpnn_trans_demo/utils/metrics.py:41: RuntimeWarning: divide by zero encountered in true_divide
  return SSReg / SST
/home/Tabe/Documents/Pro/Mpnn_trans_demo/utils/metrics.py:41: RuntimeWarning: divide by zero encountered in true_divide
  return SSReg / SST
/home/Tabe/Documents/Pro/Mpnn_trans_demo/utils/metrics.py:41: RuntimeWarning: divide by zero encountered in true_divide
  return SSReg / SST
/home/Tabe/Documents/Pro/Mpnn_trans_demo/utils/metrics.py:41: RuntimeWarning: divide by zero encounter

In [19]:
print(sum(mae_list) / len(mae_list))
print(sum(rmse_list) / len(rmse_list))


27.716535107506086
31.878084811877493
